# Asynchronous Web Scraping Python

## Learning Outcomes

- To understand the benefits of using async + await compared to simply web scraping with the [requests library.](https://requests.readthedocs.io/en/master/)
- Learn how to create an asynchronous web scraper from scratch in pure python using [asyncio](https://docs.python.org/3/library/asyncio.html) and [aiohttp](https://docs.aiohttp.org/).
- Practice downloading multiple webpages using Aiohttp + Asyncio and parsing HTML content per URL with [BeautifulSoup.](https://www.crummy.com/software/BeautifulSoup/doc)

-----------------------------------------------------------------

The following python installations are for a Jupyter Notebook, however if you are using a command line then simply <strong> exclude the ! symbol </strong>

~~~
!pip install beautifulsoup4
!pip install requests
!pip install aiohttp

~~~

In [1]:
# Library Imports
import aiohttp
import nest_asyncio
from bs4 import BeautifulSoup
import pandas as pd
import requests

--------------------------------------------------------

<strong> Note: The only reason why we use nest_asyncio is because this tutorial is written in a jupyter notebook, however if you wanted to write the same web scraper code in a python file, then you would'nt need to install or run the following code block:

In [7]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

----------------------------------------

## Why Use Asychronous Web Scraping?

Writing synchronous web scrapers are easier and the code is less complex, however they're incredibly slow. 

This is because all of the requests must wait for the current request to finish one by one. <strong> There can only be one request running at a given time. </strong>


In contrast, asynchronous web requests are able to execute without depending on previous requests within a queue or for loop. <strong> Asychronous requests happen simultaneously. </strong>

![asychronous vs synchronous computer science diagram](https://sempioneer.com/wp-content/uploads/2020/12/asychronous-vs-synchronous.jpg)

--------------------------------------------------------

## How Is Asychronous Web Scraping Different To Using Python Requests?

Instead of thinking about creating a for loop with Xn requests, you need to think about creating an event loop. For example the environment for [NodeJS](https://nodejs.org/en/), by design executes in a single threaded event loop.

However for Python, we will manually create an event loop with [asyncio.](https://docs.python.org/3/library/asyncio.html)

Inside of your event loop, you can set a number of tasks to be completed and every task will be created and executed asychronously.

------------------------------------------------------------

## How To Web Scrape A Single Web Page Using Aiohttp

In [10]:
import aiohttp
import asyncio

async def main():
    
    async with aiohttp.ClientSession() as session:
        async with session.get('http://python.org') as response:
            text = await response.text()
            some_cleaned_data = await function_name(text)
            print('Test')
            print(text)
    
asyncio.run(main())

Test
<!doctype html>
<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->
<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->
<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->
<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">
    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jqueryui/1.12.1/jquery-ui.min.js">

    <meta name="application-name" content="Python.org">
    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">
    <meta name="apple-mobile-web-app-title" content="Python.org">
    <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">

    <meta n

In [5]:
main()

<coroutine object main at 0x7fc92cc5e170>

Firstly we define a client session with aiohttp:

~~~
async with aiohttp.ClientSession() as session:

~~~

Then with our session, we execute a get response on a single URL:

~~~
async with session.get('http://python.org') as response:     
        
 ~~~
 
 Thirdly, notice how we use the await keyword in front of response.text() like this:
 
~~~
html = await response.text()

~~~

Also note that every asychronous function starts with:

~~~
async def function_name
~~~

----

Finally we run asyncio.run(main()), this creates an event loop and executes all tasks within it. 

After all of the tasks have been completed then the event loop is automatically destroyed.

------------------------------------------------------------------------------------

## How To Web Scrape Multiple Pages Using Aiohttp

When scraping multiple pages with asyncio and aiohttp, we'll use the following pattern to create multiple tasks that will be simulataneously executed within an asyncio event loop:

~~~
tasks = []

for url in urls:
    tasks.append(some_function(session, url))
    
htmls = await asyncio.gather(*tasks)

~~~

To start with we create an empty list and then for every URL, we will attach an uncalled/uninvoked function, an AioHTTP session and the URL to the list.

The asyncio.gather(*tasks), basically tells asyncio to keep running the event loop until all of these functions within the python have been completed. It will return a list that is the same length as the number of functions (unless one of the functions within the list returned zero results).

---------------------------------------------------------------

Now that we know how to create and execute multiple tasks, let's see this in action:

In [47]:
class WebScraper(object):
    def __init__(self, urls):
        self.urls = urls
        # Global Place To Store The Data:
        self.all_data  = []
        self.master_dict = {}
        # Run The Scraper:
        asyncio.run(self.main())

    async def fetch(self, session, url):
        try:
            async with session.get(url) as response:
                text = await response.text()
                return text, url
        except Exception as e:
            print(str(e))

    async def main(self):
        tasks = []
        headers = {
            "user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
        async with aiohttp.ClientSession(headers=headers) as session:
            for url in self.urls:
                tasks.append(self.fetch(session, url))

            htmls = await asyncio.gather(*tasks)
            self.all_data.extend(htmls)

            # Storing the raw HTML data.
            for html in htmls:
                if html is not None:
                    url = html[1]
                    self.master_dict[url] = {'Raw Html': html[0]}
                else:
                    continue


In [48]:
# 1. Create a list of URLs for our scraper to get the data for:
urls = ['https://understandingdata.com/', 'http://twitter.com/']

In [49]:
# 2. Create the scraper class instance, this will automatically create a new event loop within the __init__ method:
scraper = WebScraper(urls = urls)

In [50]:
# 3. Notice how we have a list length of 2:
len(scraper.all_data)

2

------------------------------------------------------------------------

## Adding HTML Parsing Logic To The Aiohttp Web Scraper

As well as collecting the HTML response from multiple webpages, parsing the web page can be useful for SEO and HTML Content Analysis.

Therefore let's create second function which will parse the HTML page and will extract the title tag.

~~~

!pip install beautifulsoup

~~~

In [51]:
class WebScraper(object):
    def __init__(self, urls):
        self.urls = urls
        # Global Place To Store The Data:
        self.all_data  = []
        self.master_dict = {}
        # Run The Scraper:
        asyncio.run(self.main())

    async def fetch(self, session, url):
        try:
            async with session.get(url) as response:
                # 1. Extracting the Text:
                text = await response.text()
                # 2. Extracting the <title> </title> Tag:
                title_tag = await self.extract_title_tag(text)
                return text, url, title_tag
        except Exception as e:
            print(str(e))
            
    async def extract_title_tag(self, text):
        try:
            soup = BeautifulSoup(text, 'html.parser')
            return soup.title
        except Exception as e:
            print(str(e))

    async def main(self):
        tasks = []
        headers = {
            "user-agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
        async with aiohttp.ClientSession(headers=headers) as session:
            for url in self.urls:
                tasks.append(self.fetch(session, url))

            htmls = await asyncio.gather(*tasks)
            self.all_data.extend(htmls)

            # Storing the raw HTML data.
            for html in htmls:
                if html is not None:
                    url = html[1]
                    self.master_dict[url] = {'Raw Html': html[0], 'Title': html[2]}
                else:
                    continue


In [52]:
scraper = WebScraper(urls = urls)

In [53]:
scraper.master_dict['https://understandingdata.com/']['Title']

<title>understandingdata.com | 520: Web server is returning an unknown error</title>

------------------------------------------------------------------

## Conclusion

Asynchronous web scraping is more suitable when you have a larger number of URLs that need to be processed quickly. 

Also you can see how easy it is to add on a HTML parsing function with BeautifulSoup, allowing you to easily extract specific elements on a per URL basis.